In [1]:
# Install the yfinance library:
pip install yfinance

     ------------------------------------ 102.2/102.2 kB 978.8 kB/s eta 0:00:00
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl (11.6 MB)
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
     ------------------------------------- 948.2/948.2 kB 12.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)
     ---------------------------------------- 3.8/3.8 MB 14.3 MB/s eta 0:00:00
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
  Created wheel for peewee: filename=peewee-3.17.8-py3-none-any.whl size=


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import yfinance as yf
import pandas as pd

# Define the tickers for S&P 500 (^GSPC) and VIX (^VIX)
tickers = ["^GSPC", "^VIX"]

# Download weekly data from Yahoo Finance
data = yf.download(tickers, start="2014-12-01", end="2024-11-30", interval="1wk", group_by="ticker")

# Extract closing prices
sp500_close = data["^GSPC"]["Close"]
vix_close = data["^VIX"]["Close"]

# Calculate weekly returns for S&P 500 (percentage change)
sp500_return = sp500_close.pct_change() * 100

# Combine data into a single DataFrame
sp500_vix = pd.DataFrame({
    "Date": sp500_close.index,
    "SP500_Return (%)": sp500_return,
    "VIX_Close": vix_close
}).set_index("Date")

# Drop rows with NaN values (e.g., first row for returns calculation)
sp500_vix.dropna(inplace=True)

# Save to a CSV file
#combined_data.to_csv("weekly_sp500_vix.csv")

sp500_vix

[*********************100%***********************]  2 of 2 completed


,SP500_Return (%),VIX_Close
Date,,
2014-12-08,-3.519380,21.080000
2014-12-15,3.412022,16.490000
2014-12-22,0.875093,14.500000
2014-12-29,-1.463544,17.790001
2015-01-05,-0.650563,17.549999
...,...,...
2024-10-28,-1.365680,21.879999
2024-11-04,4.656128,14.940000
2024-11-11,-2.083547,16.139999
